In [6]:
# coding: utf-8

import networkx as nx
import pandas as pd
import numpy as np
from operator import attrgetter

from vresutils.costdata import annuity

import pypsa


In [8]:
# rule base_network:
#     input:
#         buses='resources/buses.csv',
#         lines='resources/lines.csv',
#         population='resources/population.csv'
#     output: "networks/base_{opts}.nc"
#     benchmark: "benchmarks/base_network_{opts}"
#     threads: 1
#     resources: mem_mb=1000
#     script: "scripts/base_network.py"

import yaml
with open("../config.yaml", 'r') as stream:
    try:
        snakemake_config=yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
snakemake_input_buses='../resources/buses_2.csv'
snakemake_input_lines='../resources/lines_2.csv'
snakemake_input_population='../resources/population.csv'
snakemake_wildcards_opts = 'LC'

In [16]:
n = pypsa.Network()
n.name = 'PyPSA-ZA'

buses = pd.read_csv(snakemake_input_buses, index_col=0)
lines = pd.read_csv(snakemake_input_lines, index_col=0)

buses.loc['RSA','population'] = pd.read_csv(snakemake_input_population, index_col=0)['population'].sum()

line_config = snakemake_config['lines']
v_nom = line_config['v_nom']
line_type = line_config['type']

lines['capacity'] = np.sqrt(3)*v_nom*n.line_types.loc[line_type, 'i_nom']*lines.num_parallel

# Buses from regions
n.set_snapshots(pd.date_range(snakemake_config['historical_year'], periods=8760, freq='h'))
n.import_components_from_dataframe(buses, 'Bus')

if 'T' in snakemake_wildcards_opts.split('-'):
    n.import_components_from_dataframe(
        (lines
            .drop('num_parallel', axis=1)
            .rename(columns={'capacity': 'p_nom_min'})
            .assign(p_nom_extendable=True, p_min_pu=-1)),
        "Link"
    )
elif 'FL' in snakemake_wildcards_opts.split('-'):
    n.import_components_from_dataframe(
        (lines
            .loc[lines.num_parallel > 0.1]
            .drop('capacity', axis=1)
            .assign(s_nom_extendable=False, type=line_type)),
        "Line"
    )
else:
    n.import_components_from_dataframe(
        (lines
            .rename(columns={'capacity': 's_nom_min'})
            .assign(s_nom_extendable=True, type=line_type,
                    num_parallel=lambda df: df.num_parallel.clip(lower=0.5))),
        "Line"
    )



In [19]:
n.export_to_netcdf('../networks/base_LC.nc')

INFO:pypsa.io:Exported network base_LC.nc has buses


<xarray.Dataset>
Dimensions:                       (snapshots: 8760, investment_periods: 0,
                                   buses_i: 1)
Coordinates:
  * snapshots                     (snapshots) int64 0 1 2 3 ... 8757 8758 8759
  * investment_periods            (investment_periods) object 
  * buses_i                       (buses_i) object 'RSA'
Data variables:
    snapshots_snapshot            (snapshots) datetime64[ns] 2012-01-01 ... 2...
    snapshots_objective           (snapshots) float64 1.0 1.0 1.0 ... 1.0 1.0
    snapshots_stores              (snapshots) float64 1.0 1.0 1.0 ... 1.0 1.0
    snapshots_generators          (snapshots) float64 1.0 1.0 1.0 ... 1.0 1.0
    investment_periods_objective  (investment_periods) object 
    investment_periods_years      (investment_periods) object 
    buses_x                       (buses_i) float64 19.45
    buses_y                       (buses_i) float64 -28.96
    buses_v_nom                   (buses_i) float64 380.0
    buses_population              (buses_i) float64 5.137e+07
Attributes:
    network_name:           PyPSA-ZA
    network_pypsa_version:  0.20.0
    network_srid:           4326
    meta:                   {}